## Data Parsing

In [1]:
import os
import re
import time
import json
import httpx
import random
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [3]:
data_dir = "../data/"
raw_dir = os.path.join(data_dir, "raw")
raw_csv_dir = os.path.join(raw_dir, "csv")
raw_json_dir = os.path.join(raw_dir, "json")
html_dir = os.path.join(raw_dir, "html")
completed_html_dir = os.path.join(html_dir, "completed")
upcoming_html_dir = os.path.join(html_dir, "upcoming")
fighterlist_html_dir = os.path.join(html_dir, "fighterlist")
fighters_html_dir = os.path.join(html_dir, "fighters")
completed_eventlist_html_dir = os.path.join(completed_html_dir, "eventlist")
completed_events_html_dir = os.path.join(completed_html_dir, "events")
completed_fights_html_dir = os.path.join(completed_html_dir, "fights")
upcoming_eventlist_html_dir = os.path.join(upcoming_html_dir, "eventlist")
upcoming_events_html_dir = os.path.join(upcoming_html_dir, "events")
upcoming_fights_html_dir = os.path.join(upcoming_html_dir, "fights")

In [4]:
dirs = [raw_csv_dir, raw_json_dir, fighters_html_dir, fighterlist_html_dir,
        completed_eventlist_html_dir, 
        completed_events_html_dir,
        upcoming_eventlist_html_dir, upcoming_events_html_dir,
       completed_fights_html_dir, upcoming_fights_html_dir]

for folderpath in dirs:
    os.makedirs(folderpath, exist_ok=True)

### Parse Fight page

In [5]:
def indiv_fight_data_extractor(fight_id_html):
    fight_id, fight_html = fight_id_html 
    fight_dict = {}

    soup = BeautifulSoup(fight_html)
    title_a_elem = soup.find("h2", class_="b-content__title").find("a")
    fight_dict["Event Name"] = title_a_elem.text.strip()
    fight_dict["Event Url"] = title_a_elem["href"]
    fight_dict["Fight ID"] = fight_id
    fighters = soup.find_all("div", class_="b-fight-details__person")
    for idx, fighter in enumerate(fighters, start=1):
        fight_dict[f"Fighter{idx} Status"] = fighter.find("i", class_="b-fight-details__person-status").text.strip()

        a_elem = fighter.find("a", class_="b-link b-fight-details__person-link")
        fight_dict[f"Fighter{idx} Name"] = a_elem.text.strip()
        fight_dict[f"Fighter{idx} Url"] = a_elem["href"]

    fight_dict["Bout"] = soup.find("i", class_="b-fight-details__fight-title").text.strip()

    fight_details_div = soup.find("div", class_="b-fight-details__content")

    method_elem = fight_details_div.find("p").find("i", class_="b-fight-details__text-item_first")

    def get_details(i_elem):
        i_text = i_elem.text.replace("\n", "").strip()
        m = re.search(r"(.*):\s+(.*)", i_text)
        if m:
            return (m.group(1), m.group(2))

    label_elems = fight_details_div.find_all("i", class_="b-fight-details__label")
    detail_elems = lmap(lambda e: e.parent, label_elems)
    detail_tups = lfilter(lambda t: t != None, map(get_details, detail_elems))

    for label, text in detail_tups:
        fight_dict[label] = text

    fight_dict

    details_text = fight_details_div.find_all("p")[1].text.replace("\n", "").strip()

    m = re.search(r"(.*):\s+(.*)", details_text)
    if m:
        fight_dict[m.group(1)] = m.group(2)

    fight_dict

    tables = soup.find_all("table")

    len(tables)

    def fight_tables_to_dicts(page_html: str):

        def extract_table(tables):
            data_dict = {key:[] for key in map(lambda x: x.text.strip(), tables[0].find("thead").find_all("th"))}
            data_dict["Round"] = []

            for i, table in enumerate(tables[:2]):
                rows = table.find("tbody").find_all("tr")

                for j, row in enumerate(rows, start=1):
                    for col, elem in zip(data_dict.keys(), row.find_all("td")):
                        if col == "Fighter":
                            for a_elem in elem.find_all("a"):
                                data_dict[col].append(a_elem.text.strip())
                        else:
                            for p_elem in elem.find_all("p"):
                                val = p_elem.text.strip()
                                val = val if val != "---" else None
                                data_dict[col].append(val)

                    if i == 0:
                        data_dict["Round"].extend(["Overall", "Overall"])
                    else:
                        data_dict["Round"].extend([f"Round {j}", f"Round {j}"])


            return data_dict


        soup = BeautifulSoup(page_html, features="lxml")
        tables = soup.find_all("table")
        
        return extract_table(tables[:2]), extract_table(tables[2:])
         

    try:
        fight_dict["Totals"], fight_dict["Significant Strikes"] = fight_tables_to_dicts(fight_html)
    except IndexError:
        print("Table IndexError")
        pass 
    
    return fight_dict

In [6]:
def all_fight_data_extractor(fights_html_dir):
    fights_html_dict = {}

    fight_files = lfilter(lambda x: x.endswith(".html"), os.listdir(fights_html_dir))
    for filename in tqdm(fight_files):
        filepath = os.path.join(fights_html_dir, filename)
        with open(filepath, "r") as f:
            html_str = f.read()
            fight_id = filename.replace(".html","")
            fights_html_dict[fight_id] = html_str

    # fights_dict_list = []
    
    # for fight_id, fight_html in tqdm(fights_html_dict.items()):
    #     fights_dict_list.append(indiv_fight_data_extractor(fight_id, fight_html))
    
    with Pool(16) as p:
        fights_dict_list = p.map(indiv_fight_data_extractor, fights_html_dict.items())
        
    return fights_dict_list

In [157]:
completed_fights_dict_list = all_fight_data_extractor(completed_fights_html_dir)

100%|██████████| 6875/6875 [00:00<00:00, 26644.24it/s]


Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError


In [158]:
len(completed_fights_dict_list)

6875

In [159]:
filepath = os.path.join(raw_json_dir, "completed_fights_data.json")
with open(filepath, "w") as f:
    json.dump(completed_fights_dict_list, f, indent=4)

### Parse Fighter pages

In [85]:
def extract_fighter_data(fighter_page_filepath):

    with open(fighter_page_filepath, "r") as f:
        html_str = f.read()

    soup = BeautifulSoup(html_str)

    fighter_dict = dict()

    fighter_dict["Name"] = soup.find("span", class_="b-content__title-highlight").text.strip() 

    fighter_dict["Fighter ID"] = os.path.split(fighter_page_filepath)[1].replace(".html","")
    
    fighter_dict["Record"] = soup.find("span", class_="b-content__title-record").text.replace("Record:","").strip() 

    nn = soup.find("p", class_="b-content__Nickname").text.strip()
    fighter_dict["Nickname"] = nn if nn else None

    fighter_dict

    uls = soup.find_all("ul", class_="b-list__box-list")

    info_list = [li.text.replace("\n","").strip() for ul in uls for li in ul.find_all("li")]

    info_list = lfilter(lambda s: s, info_list)

    for info in info_list:
        m = re.search(r"(.*):\s+(.*)", info)
        if m:
            key, val = m.group(1).strip(), m.group(2).strip()
            if key == "Height":
                val = val.replace(" ","")
            if val == "--":
                val = None
            fighter_dict[key] = val

    return fighter_dict

In [86]:
def create_fighters_df(fighters_html_dir):
    fighter_files = os.listdir(fighters_html_dir)
    fighter_files = lfilter(lambda s: s.endswith(".html"), fighter_files)
    fighter_filepaths = lmap(lambda s: os.path.join(fighters_html_dir, s), fighter_files)
    
    with Pool(16) as p:
        fighters_lod = p.map(extract_fighter_data, fighter_filepaths)
        
    return pd.DataFrame(fighters_lod)

In [89]:
fighters_df = create_fighters_df(fighters_html_dir)

In [90]:
fighters_df

,Name,Fighter ID,Record,Nickname,Height,Weight,Reach,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Erik Apple,1ccff7f0cfdf85eb,10-3-0,None,"6'0""",170 lbs.,None,Orthodox,"Aug 26, 1977",0.00,0%,0.54,81%,0.00,0%,0%,8.1
1,Israel Albuquerque,dd1b90eea08887f6,0-3-0,None,None,185 lbs.,None,Orthodox,None,0.32,13%,3.34,25%,0.00,0%,0%,0.0
2,Azunna Anyanwu,78d48d3874dacafd,14-5-0,Zulu,"6'1""",251 lbs.,None,Orthodox,"Aug 05, 1981",2.47,42%,2.47,69%,0.65,100%,0%,0.0
3,Tom Aspinall,399afbabc02376b5,12-3-0,None,"6'5""",256 lbs.,"78""",Orthodox,"Apr 11, 1993",7.41,65%,2.87,64%,4.00,100%,100%,2.0
4,Nariman Abbasov,59a9d6dac61c2540,28-4-0,Bayraktar,"5'8""",155 lbs.,"66""",Orthodox,"Feb 01, 1994",3.00,20%,5.67,46%,0.00,0%,66%,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Jose Aldo,d0f3959b4a9747e6,31-8-0,None,"5'7""",135 lbs.,"70""",Orthodox,"Sep 09, 1986",3.57,46%,3.74,60%,0.49,54%,91%,0.1
210,Rico Altamirano,c61b49cec6abd6b9,4-3-0,None,"5'10""",170 lbs.,None,NaN,None,0.00,0%,0.00,0%,0.00,0%,0%,0.0
211,Askar Askar,2abc414136504554,11-1-0,AK-47,"5'7""",145 lbs.,"69""",Orthodox,"Aug 19, 1994",0.00,0%,0.00,0%,0.00,0%,0%,0.0
212,CJ Marsh,4c91eb190d5e0dd9,5-7-0,None,None,None,None,NaN,None,0.00,0%,0.00,0%,0.00,0%,0%,0.0


In [91]:
filepath = os.path.join(raw_csv_dir, "fighters_data.csv")
fighters_df.to_csv(filepath, index=False)